# Convolutional Neural Network

### Importing the libraries

In [34]:
#!pip install --upgrade tensorflow keras Pillow

In [35]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [36]:
tf.__version__

'2.6.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [37]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


### Preprocessing the Test set

In [38]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 2000 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [39]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [40]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [41]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [42]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a third convolutional layer

In [43]:
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a fourth convolutional layer

In [44]:

#cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu'))
#cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [45]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [46]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [47]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [53]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [54]:
%%time

cnn_fit = cnn.fit(x = training_set, validation_data = test_set, epochs = 15)

Epoch 1/15
250/250 [==============================] - 38s 150ms/step - loss: 0.4541 - accuracy: 0.7853 - val_loss: 0.4207 - val_accuracy: 0.8050
Epoch 2/15
250/250 [==============================] - 38s 152ms/step - loss: 0.4370 - accuracy: 0.8010 - val_loss: 0.4220 - val_accuracy: 0.8040
Epoch 3/15
250/250 [==============================] - 38s 153ms/step - loss: 0.4136 - accuracy: 0.8058 - val_loss: 0.4233 - val_accuracy: 0.7980
Epoch 4/15
250/250 [==============================] - 37s 148ms/step - loss: 0.3954 - accuracy: 0.8151 - val_loss: 0.4007 - val_accuracy: 0.8220
Epoch 5/15
250/250 [==============================] - 38s 151ms/step - loss: 0.3925 - accuracy: 0.8199 - val_loss: 0.4033 - val_accuracy: 0.8140
Epoch 6/15
250/250 [==============================] - 37s 149ms/step - loss: 0.3769 - accuracy: 0.8289 - val_loss: 0.4110 - val_accuracy: 0.8185
Epoch 7/15
250/250 [==============================] - 38s 152ms/step - loss: 0.3665 - accuracy: 0.8381 - val_loss: 0.4031 - val_ac

## Part 4 - Making a single prediction

In [101]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/single_prediction/cat_or_dog_1.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] >= 0.5:
  prediction = 'dog'
else:
  prediction = 'cat'


In [102]:
print(prediction)

dog


In [105]:
test_pred = cnn.predict(test_set)

classification = np.where(test_pred>=0.5, 'dog', 'cat')

print(classification)

[['dog']
 ['dog']
 ['dog']
 ...
 ['cat']
 ['cat']
 ['cat']]


## Accuracy mean

In [106]:
import statistics

accuracy = cnn_fit.history['val_accuracy']

print(statistics.mean(accuracy))

0.8268333355585734


## Fim